In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import time
import numpy as np

In [2]:
df1 = pd.read_csv("../csv/meneame_scraped_final_1.csv", encoding="utf-8")
df2 = pd.read_csv("../csv/meneame_scraped_final_2.csv", encoding="utf-8")
df3 = pd.read_csv("../csv/meneame_scraped_final_3.csv", encoding="utf-8")

FileNotFoundError: [Errno 2] No such file or directory: '../csv/meneame_scraped_final_1.csv'

In [ ]:
df = pd.concat([df1,df2,df3], ignore_index=True)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287191 entries, 0 to 287190
Data columns (total 19 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   news_id          287191 non-null  int64 
 1   title            287191 non-null  object
 2   content          287191 non-null  object
 3   full_story_link  287191 non-null  object
 4   meneos           287191 non-null  int64 
 5   clicks           287191 non-null  int64 
 6   karma            287191 non-null  int64 
 7   positive_votes   287191 non-null  int64 
 8   anonymous_votes  287191 non-null  int64 
 9   negative_votes   287191 non-null  int64 
 10  category         287191 non-null  object
 11  comments         287191 non-null  int64 
 12  published_date   287191 non-null  object
 13  user             287191 non-null  object
 14  source           287191 non-null  object
 15  source_link      287191 non-null  object
 16  provincia        287191 non-null  object
 17  comunidad 

In [ ]:
df["published_date"] = pd.to_datetime(df["published_date"])
df["scraped_date"] = pd.to_datetime(df["scraped_date"])

In [ ]:
df = df.drop_duplicates(subset=["news_id"])

# Procesamiento de 'comunidad'

In [ ]:
# Diccionario de provincias-ccaa
PROVINCIAS_COMUNIDADES = {
    "Andalucía": {
        "provincias": ["Almería", "Cádiz", "Córdoba", "Granada", "Huelva", "Jaén", "Málaga", "Sevilla"]
    },
    "Aragón": {
        "provincias": ["Huesca", "Teruel", "Zaragoza"]
    },
    "Asturias": {
        "provincias": ["Asturias"]
    },
    "Islas Baleares": {
        "provincias": ["Illes Balears"],
        "equivalencias": {
            "Mallorca": "Illes Balears",
            "Menorca": "Illes Balears",
            "Cabrera": "Illes Balears",
            "Ibiza": "Illes Balears",
            "Formentera": "Illes Balears"
        }
    },
    "Canarias": {
        "provincias": ["Las Palmas", "Santa Cruz de Tenerife"],
        "equivalencias": {
            "La Gomera": "Canarias",
            "El Hierro": "Canarias",
            "La Graciosa": "Canarias",
            "La Palma": "Canarias",
            "Gran Canaria": "Canarias",
            "Tenerife": "Canarias"
            "Lanzarote": "Canarias"
            "Fuerteventura": "Canarias"
        }
    },
    "Cantabria": {
        "provincias": ["Cantabria"]
    },
    "Castilla La Mancha": {
        "provincias": ["Albacete", "Ciudad Real", "Cuenca", "Guadalajara", "Toledo"]
    },
    "Castilla y León": {
        "provincias": ["Ávila", "Burgos", "León", "Palencia", "Salamanca", "Segovia", "Soria", "Valladolid", "Zamora"]
    },
    "Cataluña": {
        "provincias": ["Barcelona", "Girona", "Lleida", "Tarragona"],
        "equivalencias": {
            "Gerona": "Girona",
            "Lérida": "Lleida"
        }
    },
    "Extremadura": {
        "provincias": ["Badajoz", "Cáceres"]
    },
    "Galicia": {
        "provincias": ["A Coruña", "Lugo", "Ourense", "Pontevedra"],
        "equivalencias": {
            "Coruña": "A Coruña"
        }
    },
    "Madrid": {
        "provincias": ["Madrid"]
    },
    "Murcia": {
        "provincias": ["Murcia"]
    },
    "Navarra": {
        "provincias": ["Navarra"]
    },
    "La Rioja": {
        "provincias": ["La Rioja"]
    },
    "País Vasco": {
        "provincias": ["Araba/Álava", "Bizkaia/Vizcaya", "Gipuzkoa/Guipúzcoa"],
        "equivalencias": {
            "Araba": "Araba/Álava",
            "Álava": "Araba/Álava",
            "Bizkaia": "Bizkaia/Vizcaya",
            "Vizcaya": "Bizkaia/Vizcaya",
            "Gipuzkoa": "Gipuzkoa/Guipúzcoa",
            "Guipúzcoa": "Gipuzkoa/Guipúzcoa"
        }
    },
    "Comunidad Valenciana": {
        "provincias": ["Alicante/Alacant", "Castelló/Castellón", "València/Valencia"],
        "equivalencias": {
            "Alicante": "Alicante/Alacant",
            "Alacant": "Alicante/Alacant",
            "Castellón": "Castelló/Castellón",
            "Castelló": "Castelló/Castellón",
            "Valencia": "València/Valencia",
            "València": "València/Valencia"
        }
    },
    "Ceuta": {
        "provincias": ["Ceuta"]
    },
    "Melilla": {
        "provincias": ["Melilla"]
    }
}

In [ ]:
# función para asignar provincias y ccaa del title/content
def asignar_provincia_comunidad(row):
    for comunidad, datos in PROVINCIAS_COMUNIDADES.items():
        provincias = datos["provincias"]
        equivalencias = datos.get("equivalencias", {})
        
        for provincia in provincias:
            if provincia in row["title"] or provincia in row["content"]:
                return {"provincia": provincia, "comunidad": comunidad}
        
        for variante, provincia_estandar in equivalencias.items():
            if variante in row["title"] or variante in row["content"]:
                return {"provincia": provincia_estandar, "comunidad": comunidad}

    return {"provincia": "Desconocido", "comunidad": "Desconocido"}

In [ ]:
# Aplicar la función fila por fila
df[["provincia", "comunidad"]] = df.apply(asignar_provincia_comunidad, axis=1).apply(pd.Series)

In [ ]:
df["comunidad"].value_counts()

comunidad
Madrid                  14402
Andalucía                6524
Comunidad Valenciana     5534
Cataluña                 5443
Castilla y León          3923
Castilla La Mancha       1588
Galicia                  1567
Aragón                   1366
Murcia                   1234
Asturias                 1138
Islas Baleares           1036
Navarra                   540
País Vasco                532
Extremadura               489
Cantabria                 408
Canarias                  298
Ceuta                     240
Melilla                   158
La Rioja                  146
Name: count, dtype: int64

In [ ]:
df['comunidad'] = df['comunidad'].replace('Desconocido', np.nan)

df['provincia'] = df['provincia'].replace('Desconocido', np.nan)



# Procesamiento de Categorias


In [ ]:
news_category_map = {
    "Geopolítica y Sociedad": ["EEUU", "MADRID", "Rusia", "Japón", "México", "Venezuela", "Cataluña", "Galiza", "Andalucía", "Balears", "Canarias", "CastillaLeón", "deutschland", "Barcelona", "Galicia", "Cantabria", "Aragón", "Málaga", "Roma", "INTERNAC", "Africa", "OrienteMedio", "Cuba"],
    "Movimientos políticos": ["politica", "corruPPción", "corruptcion", "corrupcion", "EleccionesE", "Podemos", "República", "RentaBásica", "Cayetanos", "LuchaDClases", "Anarquismo", "DDHH", "Autonomos", "Justicia7291", "Tribunales", "actualidad", "ZasSmeame", "Campañas", "Fachéame", "VIOLENCIA_G", "prensa", "borbones", "KKKantiPDMS", "AEDE", "procescatalà", "RebeliónMnm", "DchoLaboral", "Bulos", "Desinfórmame", "Madriléame", "Begoñéame"],
    "Conflictos y Guerra": ["Palestina", "Israel", "Siria", "GuerraCivilñ", "ayudaucrania", "Rescates"],
    "Tecnología y Ciencia": ["tecnología", "Informática", "GNU_LINUX", "linux", "startup", "Criptomoneda", "bitcoin", "SectorTIC", "Meteorología", "Física", "genetica", "Neurociencia", "Debian", "retrocomp", "hacking", "Metamaterial", "Micromundo", "atomo", "Impresión3D", "Drones", "netsec", "RaspberryPi", "SysDevs", "ciencia", "astronomia", "ASTRONÁUTICA", "softlibre", "Ubuntu", "android", "smartphone", "Firefox", "Blender", "IOOcia", "Marte", "Visualdata", "INVENTOS", "Numismática", "FÓSILES"],
    "Salud y Medicina": ["salud", "medicina", "Nutrición", "sanidad", "PREVENCIÓN", "HOMEOPATÍA", "VacunaGate", "vacúname", "SANIDAD", "Alimentación", "ebola", "Coronavirus"],
    "Gastronomía": ["Cachopos", "cervecéame"],
    "Negocios y Economía": ["Economía", "banca", "subvenciones", "CrisEnergéti"],
    "Entretenimiento y Cultura": ["ocio", "Cine", "Series", "Videojuegos", "Fotografía", "Música", "cinéfilos", "FolkloreÑ", "Cómics", "Webcomics", "ArteyMercado", "Arte", "Ilustracion", "clásica", "PALABROS", "Covers", "cultura", "Eurovision", "Poesia", "Bushido", "Arquitectura", "rol", "Podcasteros", "Viajar", "CiFi", "Gastronomia", "Clicómics", "Cocíname", "curiosidades", "acapela", "twitter", "reddit", "museo", "Relatocorto", "Autorrelatos", "Reportajes", "Entrevistas", "microrelatos", "Hazlotumismo", "Los12monos", "Pregúntame", "Recomiéndame", "LETREROS", "Escúchame", "Documentales", "STARWARS", "Ochenteando", "UHF", "Preguntas", "Fotomundo", "roleros", "metal", "B.S.O.", "TEMAZOS", "Comiñas", "MundoCelta", "Artículos", "sabíasque", "paléame"],
    "Deportes": ["deportes", "Natación", "Motociclismo", "ciclismo"],
    "Medioambiente y Energía": ["veganismo", "MenteAnimal", "Abandonos", "MUNDO_PROTE", "MAmbiente", "Cambioclima", "Ecología", "Energias", "sequía", "Naturaleza", "Botánica", "PLANTAS", "AgroInfo", "Vivienda", "Animales", "gatos", "Biologia", "Faros", "mascotas", "Cáñamo", "Arañas"],
    "Historia y Humanidades": ["Historia", "Mitología", "Arqueología", "Egiptologia", "Viñetas", "Hallazgos", "Asturias", "Jurídicas", "Etimologia", "Psicología", "Psicópatas", "Filosofía", "Efemérides", "Helénica", "Hemeroteca", "Fragmentos"],
    "Humor y Memes": ["Humor", "memes", "Sarcasmos", "Chorridolias", "Tendenciosos", "Ayuséame", "podríame", "Fakeame", "Fakejóo", "Idiocracia", "meneametoday", "TuMeme", "Cosasguays", "friki", "procrastrina", "NoMundoToday", "Memeneame", "Ardilléame", "Toréame", "OYOYOY", "gilipolleces", "chorradas"],
    "Transporte": ["aviacion", "aviones", "trenes", "DIRIGIBLES", "Submarino", "Motor", "Avionéame", "BARCOS", "Tractores", "movilidad", "alas"],
    "Educación": ["Educación", "idiomas", "Enseñanza", "Literatura", "Libros", "lenguas", "Matemáticas"],
    "Crimen": ["INV", "Gamonal", "Sectas", "Sucesos"],
    "Cuestiones Sociales": ["Asperger", "Crianza", "Drogolegas", "Feminismo", "LGBT", "Derecho", "SEXOLOGÍA", "mujeresenlah", "Abuso_Animal", "religion", "laicismo"]
}


In [ ]:
def category_mapping(category):
    return news_category_map.get(category, "Miscelaneo")

df["category"] = df["category"].apply(category_mapping)

In [ ]:
df["category"].value_counts()

category
Miscelaneo                   135837
Politica y Sociedad           94930
Entretenimiento y Cultura     40406
Tecnologia y Ciencia          14485
Historia y Humanidades          294
Humor y Memes                   239
Política y Sociedad             235
Transporte                      220
Medioambiente y Energia         198
Salud y Medicina                137
Crimen                           74
Cuestiones Sociales              69
Negocios y Economia              22
Educacion                        19
Deportes                         13
Conflictos y Guerra               8
Medioambiente y energía           3
Gastronomía                       2
Name: count, dtype: int64

In [ ]:
df

,news_id,title,content,full_story_link,meneos,clicks,karma,positive_votes,anonymous_votes,negative_votes,category,comments,published_date,user,source,source_link,provincia,comunidad,scraped_date
0,4029771,"Luigi Mangione, acusado de asesinar al CEO de ...",“Estoy abrumado y agradecido por todos los que...,https://meneame.net/story/luigi-mangione-acusa...,53,139,345,29,24,0,Politica y Sociedad,17,2025-02-19 07:00:13,Verdaderofalso,infobae.com,https://www.infobae.com/estados-unidos/2025/02...,NaN,NaN,2025-02-19
1,4029544,La propaganda en la educación japonesa durante...,La historia de Japón es la historia de una nac...,https://meneame.net/story/propaganda-educacion...,40,537,362,26,14,0,Entretenimiento y Cultura,1,2025-02-19 05:00:06,Charles_Dexter_Ward,newsletter.mapasmilhaud.com,https://newsletter.mapasmilhaud.com/p/propagan...,NaN,NaN,2025-02-19
2,4029786,El CEO multimillonario de Palantir no puede de...,"Alex Karp, el inquietante CEO de la inquietant...",https://meneame.net/story/ceo-multimillonario-...,104,328,398,52,52,1,Politica y Sociedad,22,2025-02-19 04:05:02,Verdaderofalso,es.gizmodo.com,https://es.gizmodo.com/el-ceo-multimillonario-...,NaN,NaN,2025-02-19
3,4029797,El creador de $LIBRA alardeaba “controlar” a J...,Uno de los mensajes a los que tuvo acceso LA N...,https://meneame.net/story/creador-libra-alarde...,103,418,446,53,50,0,Politica y Sociedad,26,2025-02-19 03:25:03,Suikoden,lanacion.com.ar,https://www.lanacion.com.ar/politica/escandalo...,NaN,NaN,2025-02-19
4,4029611,Uso de CRISPR para eliminar cromosomas adicion...,Las técnicas de edición genética podrían permi...,https://meneame.net/story/uso-crispr-eliminar-...,66,189,387,37,29,0,Tecnologia y Ciencia,15,2025-02-19 03:00:03,onainigo,eurekalert.org,https://www.eurekalert.org/news-releases/1073061,NaN,NaN,2025-02-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287186,6,Instalar Linux en iPod Mini | GPL Tarragona,Enseña a instalar Linux en el iPod Mini.\n\nMu...,https://meneame.net/story/instalar-linux-en-ip...,29,0,31,29,0,0,Miscelaneo,0,2005-12-07 12:00:01,asappla,gpltarragona.org,http://www.gpltarragona.org/node/view/395,Tarragona,Cataluña,2025-02-19
287187,4,Entrevista de El Mundo a Jose Antonio Marina,2. ¿Es la ética la filosofía del futuro?\nSi. ...,https://meneame.net/story/entrevista-de-el-mun...,18,0,22,18,0,0,Miscelaneo,0,2005-12-07 11:30:01,gabrielperezs,el-mundo.es,http://www.el-mundo.es/encuentros/invitados/20...,NaN,NaN,2025-02-19
287188,3,Prova xfce 4.2.3!,Ara que els de repositoris de totes les distri...,https://meneame.net/story/prova-xfce-423,34,0,13,34,0,0,Miscelaneo,0,2005-12-07 10:30:01,paurullan,xfce.org,http://www.xfce.org/release_notes/4.2.3.1_chan...,NaN,NaN,2025-02-19
287189,2,Los 84 errores de noxtrum,"Pues sí, noxtrum, el buscador de TPI y Telefón...",https://meneame.net/story/los-84-errores-de-no...,61,0,11,61,0,0,Miscelaneo,5,2005-12-07 09:25:02,FrIkI,pacoros.net,http://www.pacoros.net/diario/2005/12/07/los-8...,NaN,NaN,2025-02-19


In [ ]:
df.to_csv("meneame_procesado.csv", index=False, encoding="utf-8")
